In [1]:
# cite simcse
# cite sentence-transformers https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2
import pandas as pd
import numpy as np

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [3]:
#df_own = pd.read_excel("data/survey_data/labelled_data/sat_data_combined_vertical.xlsx")
df_own = pd.read_excel("../data/survey_data/labelled_data/data_labelled.xlsx")

In [4]:
#df_own = df_own[df_own['empathy_score'].isna() == False]

In [5]:
df_own

,Unnamed: 0,level_1,response,final_pred_modified,original
0,0,A01,How do you feel?,0,original
1,1,A01,How are you feeling?,0,original
2,2,A01,How are you feeling today?,0,original
3,3,A01,How are you doing?,0,original
4,4,A01,How are you feeling right now?,0,original
...,...,...,...,...,...
2363,763,J05,"Share a blog post about your success story, di...",0,paraphrased2
2364,764,J05,Write a blog post sharing your success story (...,0,paraphrased2
2365,765,J05,Let us write a blog post discussing your succe...,2,paraphrased2
2366,766,J05,"Make a blog post recording your success story,...",0,paraphrased2


In [6]:
#df['empathy_score'].replace(1, 0, inplace=True)
#df['empathy_score'].replace(2, 1, inplace=True)

In [7]:
df_own_response = df_own['response'].to_list()

In [8]:
df = df_own

In [9]:
df['final_pred_modified'].value_counts()

2    871
0    824
1    673
Name: final_pred_modified, dtype: int64

In [10]:
X = df['response']
y = df['final_pred_modified']

## Lisa Dataset

In [5]:
df = pd.read_csv("../data/compassion_energy/empathy_labelled_ds.csv")

In [7]:
df['empathy_score'] = df['empathy_score'].astype(int)

In [8]:
X = df['response']
y = df['empathy_score']

In [9]:
df['empathy_score'].value_counts()

1    388
2    381
0    331
Name: empathy_score, dtype: int64

## Splitting Dataset

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.9, random_state=0)

In [12]:
texts = list(X_train)
texts_test = list(X_test)

### SimCSE

In [48]:
import torch
from scipy.spatial.distance import cosine
from transformers import AutoModel, AutoTokenizer

In [49]:
import torch
from scipy.spatial.distance import cosine
from transformers import AutoModel, AutoTokenizer

In [50]:
# Import our models. The package will take care of downloading the models automatically
tokenizer = AutoTokenizer.from_pretrained("princeton-nlp/sup-simcse-bert-base-uncased")
model = AutoModel.from_pretrained("princeton-nlp/sup-simcse-bert-base-uncased")
#tokenizer = AutoTokenizer.from_pretrained("princeton-nlp/sup-simcse-roberta-large")
#model = AutoModel.from_pretrained("princeton-nlp/sup-simcse-roberta-large")


In [51]:
inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")

# Get the embeddings
with torch.no_grad():
    embeddings = model(**inputs, output_hidden_states=True, return_dict=True).pooler_output

In [52]:
inputs_test = tokenizer(texts_test, padding=True, truncation=True, return_tensors="pt")

# Get the embeddings
with torch.no_grad():
    embeddings_test = model(**inputs_test, output_hidden_states=True, return_dict=True).pooler_output

In [53]:
embeddings.shape

torch.Size([110, 768])

## Logistic Regression

In [54]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=0).fit(embeddings, y_train)


In [55]:
y_pred = clf.predict(embeddings_test)

In [56]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.63      0.47      0.54       302
           1       0.46      0.64      0.54       342
           2       0.79      0.67      0.72       346

    accuracy                           0.60       990
   macro avg       0.63      0.59      0.60       990
weighted avg       0.63      0.60      0.60       990



## XGBoost

In [57]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [58]:
xgb = XGBClassifier()
xgb.fit(embeddings, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, objective='multi:softprob',
              predictor='auto', random_state=0, reg_alpha=0, ...)

In [59]:
y_pred_xgb = xgb.predict(embeddings_test)

In [60]:
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.58      0.41      0.48       302
           1       0.43      0.62      0.51       342
           2       0.77      0.62      0.69       346

    accuracy                           0.56       990
   macro avg       0.59      0.55      0.56       990
weighted avg       0.59      0.56      0.56       990



### Naive Bayes

In [61]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(embeddings, y_train)
y_pred = gnb.predict(embeddings_test)

In [62]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.51      0.49      0.50       302
           1       0.42      0.45      0.43       342
           2       0.72      0.68      0.70       346

    accuracy                           0.54       990
   macro avg       0.55      0.54      0.54       990
weighted avg       0.55      0.54      0.55       990



## Write to Excel

In [612]:
y_pred_own_logit = clf.predict(embeddings_own)
y_pred_own_xgb = xgb.predict(embeddings_own)

In [613]:
#y_pred_own

In [614]:
df_own['y_pred_logit'] = y_pred_own_logit
df_own['y_pred_xgb'] = y_pred_own_xgb

In [194]:
#df_own.to_excel("data/survey_data/labelled_data/sat_data_combined_vertical_pred.xlsx")

In [182]:
df_own['y_pred'].value_counts()

0    355
2    311
1    166
Name: y_pred, dtype: int64

In [175]:
df['empathy_score'].value_counts()

1    388
2    381
0    331
Name: empathy_score, dtype: int64